In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

c:\Users\klikh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)


In [4]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32


In [6]:
def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))

    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)

    # a left upside down flipped is still a dog ;)
    image = tf.image.random_flip_left_right(image)  # 50%
    # image = tf.image.random_flip_up_down(image) #%50%

    return image, label


In [7]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


In [8]:
# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)


In [9]:
model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)

Epoch 1/5
1563/1563 - 35s - loss: 2.0740 - accuracy: 0.2261 - 35s/epoch - 23ms/step
Epoch 2/5
1563/1563 - 32s - loss: 1.8857 - accuracy: 0.3158 - 32s/epoch - 21ms/step
Epoch 3/5
1563/1563 - 32s - loss: 1.7906 - accuracy: 0.3524 - 32s/epoch - 20ms/step
Epoch 4/5
1563/1563 - 34s - loss: 1.7246 - accuracy: 0.3806 - 34s/epoch - 22ms/step
Epoch 5/5
1563/1563 - 34s - loss: 1.6704 - accuracy: 0.4025 - 34s/epoch - 22ms/step
1563/1563 [==============================] - 11s 7ms/step - loss: 1.6360 - accuracy: 0.4186


[1.6360301971435547, 0.4185999929904938]

From tensorflow version >=2.3.0 the augmentation is done in another and easy way

In [14]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=32, width=32,),
        layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
        layers.experimental.preprocessing.RandomContrast(factor=0.1,),
    ]
)


data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=32, width=32,),
        layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
        layers.experimental.preprocessing.RandomContrast(factor=0.1,),
    ]
)


model2 = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        data_augmentation,
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)


In [15]:

model2.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [16]:
model2.fit(ds_train, epochs=5, verbose=2)
model2.evaluate(ds_test)

Epoch 1/5
1563/1563 - 36s - loss: 2.3026 - accuracy: 0.1005 - 36s/epoch - 23ms/step
Epoch 2/5
1563/1563 - 32s - loss: 2.1115 - accuracy: 0.1992 - 32s/epoch - 20ms/step
Epoch 3/5
1563/1563 - 31s - loss: 1.9908 - accuracy: 0.2647 - 31s/epoch - 20ms/step
Epoch 4/5
1563/1563 - 31s - loss: 1.9046 - accuracy: 0.3024 - 31s/epoch - 20ms/step
Epoch 5/5
1563/1563 - 31s - loss: 1.8430 - accuracy: 0.3233 - 31s/epoch - 20ms/step
1563/1563 [==============================] - 10s 6ms/step - loss: 1.8337 - accuracy: 0.3289


[1.8337218761444092, 0.3289400041103363]

**NOTE**:
- this sectin/notebook are for learning purposes. so I'm avoiding learning rate for now.